# KRX 데이터 조회 - 전일급등주
- description: 가즈아
- layout: post
- categories: [경제]
- toc: true

In [1]:
#hide
!pip install pandas pandasql xlrd

In [2]:
# hide
import requests
import pandas as pd
from pandas.tseries.offsets import BDay
from io import StringIO
from io import BytesIO
import json
import datetime

왜 이렇게 만들었는지 모르겠는데 KRX는 한번 꼬아서 다운로드 받는다. 이러면 내가 못할 줄 알고?

In [3]:
def downloadKrxData(cd):
    resp = requests.post(
        'http://file.krx.co.kr/download.jspx', 
        headers={
            'User-Agent': 'Mozilla/5.0', 
            'Referer': 'http://marketdata.krx.co.kr/',
        }, 
        params={'code': cd},
    )
    tp = resp.headers['Content-Disposition'].split('.')[1]
    if tp == 'csv':
        return pd.read_csv(StringIO(resp.text))
    if tp == 'xls':
        return pd.read_excel(BytesIO(resp.content))

## KRX 휴장일을 반영한 전영업일 
- KRX 01023 휴장일 http://marketdata.krx.co.kr/mdi#document=01100305

In [4]:
def closingDays(year):
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'xls',
            'url': 'MKD/01/0110/01100305/mkd01100305_01',
            'search_bas_yy': year,
            'gridTp': 'KRX',
            'pagePath': '/contents/MKD/01/0110/01100305/MKD01100305.jsp',
        },
    )
    return downloadKrxData(resp.text)['일자 및 요일']

def getPreviousBizDay(today):
    pday = datetime.datetime.strptime(today, '%Y%m%d')
    yyyy = pday.year
    cds = closingDays(yyyy)

    while True:
        pday = pday - BDay(1)
        if yyyy != pday.year:
            yyyy = pday.year
            cds = pd.concat([cds, closingDays(yyyy)])
        if pday.strftime("%Y-%m-%d") not in cds.values:
            return pday.strftime("%Y%m%d")

## 전일 등락
- KRX 80037 전체종목 등락률 http://marketdata.krx.co.kr/mdi#document=13020102

In [5]:
def soaredStocks(dt):
    print(dt)
    resp = requests.get(
        'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx', 
        headers={'User-Agent': 'Mozilla/5.0'}, 
        params={
            'name': 'fileDown',
            'filetype': 'csv',
            'url': 'MKD/13/1302/13020102/mkd13020102',
            'ind_tp': 'ALL',
            'period_strt_dd': dt,
            'period_end_dd': dt,
            'pagePath': '/contents/MKD/13/1302/13020102/MKD13020102.jsp',
        },
    )
    return downloadKrxData(resp.text)

In [6]:
# hide
pd.set_option('display.max.colwidth', 150)

## 전 영업일 15% 이상 오른 종목

In [7]:
dt = getPreviousBizDay(datetime.datetime.now().strftime('%Y%m%d'))
df_soarings = soaredStocks(dt)

20201023


In [8]:
# hide
df_15 = df_soarings[df_soarings.등락률 >= 15].sort_values('등락률', ascending=False)
df_15['네이버주식'] = 'https://finance.naver.com/item/main.nhn?code=' + df_15['종목코드']
df_15

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
577,015260,에이엔피,"1,150","1,495",345,30.00,"1,582,346","2,268,940,840",https://finance.naver.com/item/main.nhn?code=015260
1802,141070,맥스로텍,"2,700","3,510",810,30.00,"20,128,487","67,343,771,060",https://finance.naver.com/item/main.nhn?code=141070
339,006110,삼아알미늄,"7,610","9,890","2,280",29.96,"5,352,604","51,455,736,250",https://finance.naver.com/item/main.nhn?code=006110
649,019770,서연탑메탈,"3,530","4,585","1,055",29.89,"20,234,126","88,357,264,605",https://finance.naver.com/item/main.nhn?code=019770
1490,090710,휴림로봇,753,978,225,29.88,"83,672,942","77,519,899,168",https://finance.naver.com/item/main.nhn?code=090710
317,005745,크라운해태홀딩스우,"14,950","19,400","4,450",29.77,"759,979","13,513,863,700",https://finance.naver.com/item/main.nhn?code=005745
405,00806K,대덕1우,"10,100","13,100","3,000",29.70,"619,749","7,916,368,450",https://finance.naver.com/item/main.nhn?code=00806K
2471,35320K,대덕전자1우,"8,300","9,950","1,650",19.88,"3,510,027","35,004,114,980",https://finance.naver.com/item/main.nhn?code=35320K
293,005257,녹십자홀딩스2우,"156,000","186,500","30,500",19.55,"445,639","81,659,785,500",https://finance.naver.com/item/main.nhn?code=005257
1680,115500,케이씨에스,"5,200","6,200","1,000",19.23,"13,428,460","81,895,586,470",https://finance.naver.com/item/main.nhn?code=115500


In [9]:
import pandasql as ps
ps.sqldf('''
    select a.*,
           'https://finance.naver.com/item/main.nhn?code=' || a.종목코드 as '네이버주식'
      from df_soarings a
     where 등락률 >= 15
  order by 등락률 desc
''')

,종목코드,종목명,시작일기준가(원),종료일종가(원),대비,등락률,거래량(주),거래대금(원),네이버주식
0,015260,에이엔피,"1,150","1,495",345,30.00,"1,582,346","2,268,940,840",https://finance.naver.com/item/main.nhn?code=015260
1,141070,맥스로텍,"2,700","3,510",810,30.00,"20,128,487","67,343,771,060",https://finance.naver.com/item/main.nhn?code=141070
2,006110,삼아알미늄,"7,610","9,890","2,280",29.96,"5,352,604","51,455,736,250",https://finance.naver.com/item/main.nhn?code=006110
3,019770,서연탑메탈,"3,530","4,585","1,055",29.89,"20,234,126","88,357,264,605",https://finance.naver.com/item/main.nhn?code=019770
4,090710,휴림로봇,753,978,225,29.88,"83,672,942","77,519,899,168",https://finance.naver.com/item/main.nhn?code=090710
5,005745,크라운해태홀딩스우,"14,950","19,400","4,450",29.77,"759,979","13,513,863,700",https://finance.naver.com/item/main.nhn?code=005745
6,00806K,대덕1우,"10,100","13,100","3,000",29.70,"619,749","7,916,368,450",https://finance.naver.com/item/main.nhn?code=00806K
7,35320K,대덕전자1우,"8,300","9,950","1,650",19.88,"3,510,027","35,004,114,980",https://finance.naver.com/item/main.nhn?code=35320K
8,005257,녹십자홀딩스2우,"156,000","186,500","30,500",19.55,"445,639","81,659,785,500",https://finance.naver.com/item/main.nhn?code=005257
9,115500,케이씨에스,"5,200","6,200","1,000",19.23,"13,428,460","81,895,586,470",https://finance.naver.com/item/main.nhn?code=115500
